In [ ]:
import subprocess
import shlex
import json

import numpy as np

from pathlib import Path
from IPython.display import display, Markdown

In [ ]:
target_directory = json.loads(subprocess.run("cargo metadata --format-version 1".split(), capture_output=True).stdout)["target_directory"]

In [ ]:
def shlex_split_and_print(args):
    print(args)
    
    return shlex.split(args)

In [ ]:
def simulate_cuda_reporting_dynamic(
    speciation=0.001, seed=42, sample=1.0, radius=564, sigma=10.0, report_speciation=False, report_dispersal=False, output="profile", launches=10,
):
    if report_dispersal:
        reporters = ["Execution()", "Biodiversity()", "Counter()"]
    elif report_speciation:
        reporters = ["Execution()", "Biodiversity()"]
    else:
        reporters = ["Execution()"]
        
    config = "".join(f"""
    (
        speciation: {speciation},
        sample: {sample},
        seed: {seed},

        algorithm: CUDA(
            ptx_jit: true,
            parallelism_mode: Monolithic(event_slice: {10000000}),
        ),

        scenario: AlmostInfinite(
            radius: {radius},
            sigma: {sigma},
        ),

        reporters: [
            Plugin(
                library: "{target_directory}/release/deps/libnecsim_plugins_common.so",
                reporters: [{', '.join(reporters)}]
            )
        ],
    )
    """.split()).replace(",)", ")").replace(",]", "]")
    
    # Profile the first `launches` kernel executions using NVIDIA Nsight Compute
    subprocess.run(shlex_split_and_print(
        f"/vol/cuda/10.0.130/NsightCompute-1.0/target/linux-desktop-glibc_2_11_3-glx-x64/nv-nsight-cu-cli --mode=launch-and-attach " +
        f"-k simulate -c {launches} -o ~/ml5717/compute-{output} -f {target_directory}/release/rustcoalescence simulate '{config}'"
    ), capture_output=True, text=True)
    
    # Profile the full simulation using NVIDIA Nsight Systems
    subprocess.run(shlex_split_and_print(
        f"/vol/cuda/11.1.0-cudnn8.0.4.30/nsight-systems-2020.3.4/bin/nsys profile -o ~/ml5717/nsys-{output} " +
        f"-f true -t cuda {target_directory}/release/rustcoalescence simulate '{config}'"
    ), capture_output=True, text=True)

In [ ]:
subprocess.run(shlex.split(
    f"cargo build --manifest-path {target_directory}/../rustcoalescence/Cargo.toml --release "
    + "--features rustcoalescence-algorithms-monolithic,rustcoalescence-algorithms-independent,"
    + "rustcoalescence-algorithms-cuda"
), check=True, capture_output=True, text=True);

In [ ]:
display(Markdown("# RAM Information:"))

display(Markdown('>```\n' + subprocess.run(
    shlex.split("free"), check=True, capture_output=True, text=True
).stdout.replace('\n', '\n>') + '```'))

display(Markdown("# CPU Information:"))

display(Markdown('>```\n' + subprocess.run(
    shlex.split("lscpu"), check=True, capture_output=True, text=True
).stdout.replace('\n', '\n>') + '```'))

display(Markdown("# GPU Information:"))

display(Markdown('>```\n' + subprocess.run(
    shlex.split("nvidia-smi"), check=True, capture_output=True, text=True
).stdout.replace('\n', '\n>') + '```'))

In [ ]:
seed = np.random.randint(0, np.iinfo("uint64").max, dtype="uint64")

for report_speciation, report_dispersal, output in [
    (False, False, "progress"), (True, False, "progress-speciation"), (True, True, "progress-speciation-dispersal")
]:
    simulate_cuda_reporting_dynamic(
        speciation=0.001, sample=1.0, radius=178, sigma=0.0, seed=seed, output=output,
        report_speciation=report_speciation, report_dispersal=report_dispersal,
    )